In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**READ AN AUDIO FILE**

In [ ]:
audio_file_wav = '/content/a-boy-is-reading-a-book.wav'

In [ ]:
!pip install --upgrade google-cloud-speech

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 14.3 MB/s eta 0:00:00


In [ ]:
import os
from google.cloud import speech

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'speech-to-text-382212-46fc5777a765.json'
speech_client = speech.SpeechClient()

with open(audio_file_wav, 'rb') as f1:
    byte_data = f1.read()
audio_wav = speech.RecognitionAudio(content = byte_data)

config_wav = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    audio_channel_count=2,
    enable_automatic_punctuation=True,
    language_code = 'en-US'
)

response_standard_wav = speech_client.recognize(
    config=config_wav,
    audio=audio_wav
    )

In [ ]:
audio_to_text = response_standard_wav.results[0].alternatives[0].transcript

In [ ]:
print(audio_to_text)

A boy is reading a book.


**CONVERT TEXT TO GLOSS**

In [ ]:
'''audio_to_text = 'sentence' '''

In [ ]:
# Install the most re version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!apt install --allow-change-held-packages libcudnn8=8.6.0.163-1+cuda11.8
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow
!pip install transformers --quiet
# Installing the ffmpeg
!pip install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be DOWNGRADED:
  libcudnn8
0 upgraded, 0 newly installed, 1 downgraded, 1 to remove and 22 not upgraded.
Need to get 446 MB of archives.
After this operation, 1,441 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  libcudnn8 8.6.0.163-1+cuda11.8 [446 MB]
Fetched 446 MB in 5s (85.8 MB/s)
(Reading database ... 122519 files and directories currently installed.)
Removing libcudnn8-dev (8.7.0.84-1+cuda11.8) ...
update-alternatives: removing manually selected alternative - switching libcudnn to auto mode
dpkg: warning: downgrading libcudnn8 from 8.7.0.84-1+cuda11.8 to 8.6.0.163-1+cuda11.8
(Reading database ... 122486 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.6.0

In [ ]:
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text
from tensorflow.data import AUTOTUNE
import collections
import os
import pathlib
import re
import string
import sys
import tempfile

In [ ]:
%cd /content/drive/Shareddrives/298A/Transformer_Inference

/content/drive/Shareddrives/298A/Transformer_Inference


In [ ]:
tokenizer = tf.saved_model.load('tx_gl_converter')
tokenizer.tx.get_vocab_size().numpy()

4801

In [ ]:
tx_gl_translation = tf.saved_model.load('tx-gl-translator')

In [ ]:
gloss_text = tx_gl_translation(audio_to_text).numpy()

In [ ]:
import re

# Define the pattern to match special characters
pattern = r'[^a-zA-Z0-9\s]'
# Use regular expression substitution to remove special characters
cleaned_string = re.sub(pattern, '', audio_to_text)

converted_sentence = gloss_text.decode('utf-8')  # Convert bytes to string
converted_sentence = converted_sentence.replace('x', '')  # Replace 'x' with 'you'
converted_sentence = converted_sentence.replace('desc', '')  # Replace 'desc' with 'you'
converted_sentence = converted_sentence.replace('  ', '')  # Replace double space with single space

gloss_text = converted_sentence

In [ ]:
print(gloss_text)

boy be reading book .


**GLOSS TO SKELETON VIDEO THROUGH LOOKUP**

In [ ]:
!pwd

/content/drive/Shareddrives/298A/Transformer_Inference


In [ ]:
cd /content/drive/Shareddrives/298A

/content/drive/Shareddrives/298A


In [ ]:
# Reading the Video Metadata
import pandas as pd
meta_df = pd.read_csv("video-metadata.csv")

In [ ]:
gloss_text = 'boy be reading book'
print(gloss_text)

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')  # Download the WordNet corpus

tokens = nltk.word_tokenize(gloss_text)
lemmatizer = WordNetLemmatizer()

gloss_vals = []
# Iterate over each token
for token in tokens:
  lemma = lemmatizer.lemmatize(token, pos='v')
  gloss_vals.append(lemma)

gloss_text = " ".join(gloss_vals)
print(gloss_text)

boy be reading book
boy be read book


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemma

'continue'

In [ ]:
# Defining the function to extract gloss from the gloss sequence
def get_pose_ids(gloss):
  gloss_list = gloss.split(" ")
  print(gloss_list)
  pose_ids = list()
  meta_df["Gloss Variant"] = meta_df["Gloss Variant"].apply(lambda x: x.strip("+#_()123456789"))
  for term in gloss_list:
    ids = meta_df[meta_df["Gloss Variant"] == term]["id"].values
    if len(ids) > 0:
      pose_ids.append(ids[0])
  return pose_ids

# Fetch the video clip based on the pose id
def get_pose_files(lookup_directory, pose_ids):
    pose_filepaths = []
    for pose_id in pose_ids:
        pose_filename = "pose-{}.mov".format(pose_id)
        pose_filepath = os.path.join(lookup_directory, pose_filename)
        pose_filepaths.append(pose_filepath)
    return pose_filepaths

In [ ]:
# Get posses from the gloss by calling the get_pose_ids function
extracted_poses = get_pose_ids(gloss_text.upper())
gloss_available = []
for i in extracted_poses:
  glo = meta_df[meta_df["id"] == i]["Gloss Variant"]
  gloss_available.append(glo)

gloss_available

['BOY', 'BE', 'READING', 'BOOK', '.']


[1768    BOY
 Name: Gloss Variant, dtype: object,
 1692    BOOK
 Name: Gloss Variant, dtype: object]

In [ ]:
print("Total number of gloss in a gloss sequence: ",len(gloss_text.split(" ")))
print("Number of gloss which found a mapping to video clip: ",len(extracted_poses))

Total number of gloss in a gloss sequence:  5
Number of gloss which found a mapping to video clip:  2


In [ ]:
# Fetching the video clips filepaths
video_filenames = get_pose_files('lookup/', extracted_poses)

In [ ]:
# creating a .mp4 file from .mov
import os.path
from os import path

video_filenames_mp4 = []
for every_video in video_filenames:
  input_video = every_video
  output_video = 'lookup_gloss_videolist/lookup_mp4/'+input_video.split('/')[1].split('.')[0]+'.mp4'
  video_filenames_mp4.append('lookup_mp4/'+input_video.split('/')[1].split('.')[0]+'.mp4')
  if path.exists(output_video) == False:
    print("convertion to mp4 is required for: ",input_video)
    !ffmpeg -i $input_video -b:v 1M -vcodec libx264 -preset slow -profile:v baseline -level 3.0 -pix_fmt yuv420p -movflags +faststart $output_video

In [ ]:
# Creating the content to merge the required video files
updated_gloss = gloss_text.upper().replace(" ","_")
gloss_videolist = "lookup_gloss_videolist/"+updated_gloss+".txt"
gloss_videolist_to_write = "file 'lookup_gloss_videolist/"+updated_gloss+".txt'"

#print(updated_gloss, gloss_videolist, gloss_videolist_to_write)
if path.exists(gloss_videolist) == False:
   with open(gloss_videolist, "w") as f:
      # Write each value in the list as a new line in the file
      for item in video_filenames_mp4:
        f.write("file '" + item + "'\n")

In [ ]:
# Check by reading the file in read mode
with open(gloss_videolist, "r") as f:
    file_content = f.read()
    print(file_content)

file 'lookup_mp4/pose-1768.mp4'
file 'lookup_mp4/pose-1692.mp4'



In [ ]:
#Merging all the relavant videos
merged_input = gloss_videolist
merged_output = gloss_videolist.split(".")[0]+".mp4"
#print(merged_input, merged_output)
!ffmpeg -f concat -i $merged_input -c copy $merged_output

print(merged_input)
print(merged_output)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(merged_output,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

**SKELETON VIDEO TO REALISTIC VIDEO**

In [ ]:
# Install required package
!pip uninstall dominate
!pip install dominate --quiet

Reading the video and breaking into frames

In [ ]:
import cv2
import os, shutil
merged_output = '/content/drive/Shareddrives/298A/lookup_gloss_videolist/BOY_BE_READING_BOOK_.mp4'
# Open the video file
video = cv2.VideoCapture(merged_output)
path = "/content/drive/Shareddrives/298A/frames/initial"

# Delete any previous frames in the folder
for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Check if the video file was opened successfully
if not video.isOpened():
    print("Error opening video file")
    exit()

# Set the frame number to start with
frame_num = 0

# Loop through the video frames
while True:
    # Read the next frame from the video
    ret, frame = video.read()

    # Check if the frame was read successfully
    if not ret:
        break

    # Save the frame as an image file
    # cv2.imwrite(f"frame{frame_num}.png", frame)
    cv2.imwrite(os.path.join(path , f"frame{frame_num}.png"), frame)

    # Increment the frame number
    frame_num += 1

# Release the video object
video.release()

Cropping the inferred image

In [ ]:
import os
import cv2

# Set the paths for the input folder containing images and the output folder
input_folder = '/content/drive/Shareddrives/298A/frames/initial'
output_folder = '/content/drive/Shareddrives/298A/frames/test_A'

# Delete any previous frames in the folder

for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

# Desired crop dimensions and offset
crop_width = 400
crop_height = 336
offset = 40

# Loop over the images in the input folder
for filename in os.listdir(input_folder):
    # Read the image
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)
    # Calculate the crop coordinates
    image_height, image_width = img.shape[:2]
    start_x = max(0, int((image_width - crop_width) / 2))
    start_y = max(0, int((image_height - crop_height) / 2)) - offset
    
    # Perform the crop
    cropped_image = img[start_y:start_y + crop_height, start_x:start_x + crop_width]
    
    # Create the output path for the cropped image
    output_path = os.path.join(output_folder, filename)
    
    # Save the cropped image
    cv2.imwrite(output_path, cropped_image)

In [ ]:
%cd /content/drive/Shareddrives/298A/GAN_Inference/Pose_to_Sign_GAN

Running the GAN inference on the above frames

In [ ]:
# Conduct inference on test images. Inference results will be stored in "results" directory
!python3 test.py --name pose2sign --dataroot '/content/drive/Shareddrives/298A/frames' --label_nc 0 --no_instance --resize_or_crop None --how_many 100

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: /content/drive/Shareddrives/298A/frames
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 100
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: pose2sign
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: None
results_dir: ./results/
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ----------------
CustomDatasetDataLoader
dataset 

To sort the files in alphanumeric order

In [ ]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
from os.path import isfile, join

pathIn = '/content/drive/Shareddrives/298A/GAN_Inference/Pose_to_Sign_GAN/results/pose2sign/test_latest/images'
pathOut = 'GAN_generated.mp4'
fps = 10
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
files.sort()

In [ ]:
sorted_list = sorted_alphanumeric(files)

Creating the video from inferred frames

In [ ]:
import cv2
import fnmatch

for file in sorted_list:
  if fnmatch.fnmatch(file, '*_synthesized_image.jpg'):
    filename = pathIn +'/'+ file
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

Compressing the video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD/GAN_generated.mp4"

# Compressed video path
compressed_path = "/content/drive/Shareddrives/298A/GAN_Inference/pix2pixHD/compressed_generated.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

0

Displayin ouput video

In [ ]:
# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=700 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)